In [71]:
import pandas as pd
import datetime, pytz
import random

### Read users

In [72]:
users = pd.read_excel("../../data/users_processed.xlsx")

In [73]:
users

,id,city,tz,utc_offset,team,performance,name
0,1,Kilinochchi,Asia/Colombo,530,AccountRep,0.65,Tybie Lampart
1,2,San Isidro,Asia/Manila,800,QA,0.89,Gertrud Wasselin
2,3,Liaoyang,Asia/Harbin,800,CustomerSuccess,1.09,Candide MacKissack
3,4,Novaya Mayna,Europe/Volgograd,300,QA,1.10,Alexis Sherrocks
4,5,Blimbing,Asia/Jakarta,700,QA,1.09,Jorrie Crady
...,...,...,...,...,...,...,...
1621,1996,Primero de Mayo,America/Mexico_City,-500,QA,0.92,Sylvan Wotton
1622,1997,Yuncao,Asia/Shanghai,800,AccountRep,1.06,Jerrome Pedlingham
1623,1998,Jiuzhai,Asia/Harbin,800,CustomerSuccess,1.09,Cleavland Leneham
1624,1999,Sulangan,Asia/Manila,800,Sales,0.70,Drusi Duignan


In [74]:
## Shuffle users 
users = users.sample(frac=1).reset_index(drop=True)

### Read workflow def

In [75]:
teams_df = pd.read_excel("../../data/workflow.xlsx", sheet_name="teams")
teams = teams_df['Team'].tolist()

steps_df = pd.read_excel("../../data/workflow.xlsx", sheet_name="steps")
step_names = steps_df['Step'].tolist()

users_df = pd.read_excel("../../data/users_processed.xlsx")


In [76]:
tz_list = users_df['tz'].unique()


In [77]:
tz_list

array(['Asia/Colombo', 'Asia/Manila', 'Asia/Harbin', 'Europe/Volgograd',
       'Asia/Jakarta', 'Asia/Shanghai', 'Europe/Stockholm',
       'Asia/Chongqing', 'Europe/Zaporozhye', 'Asia/Yakutsk',
       'Europe/Oslo', 'Asia/Nicosia', 'Asia/Makassar', 'Europe/Paris',
       'Europe/Uzhgorod', 'Europe/Lisbon', 'Europe/Warsaw',
       'America/Guayaquil', 'Atlantic/Cape_Verde', 'Asia/Yekaterinburg',
       'America/Port-au-Prince', 'America/Jamaica', 'America/Lima',
       'Europe/Moscow', 'Asia/Pontianak', 'Asia/Tehran', 'Europe/Tallinn',
       'America/Sao_Paulo', 'America/Mexico_City', 'Asia/Tokyo',
       'Pacific/Auckland', 'America/Mazatlan', 'Africa/Johannesburg',
       'America/Montreal', 'America/Asuncion', 'America/Phoenix',
       'America/Denver', 'Europe/Minsk', 'Asia/Sakhalin',
       'Europe/Amsterdam', 'America/Bogota', 'Asia/Irkutsk',
       'Asia/Almaty', 'America/Monterrey', 'Europe/Helsinki',
       'America/Moncton', 'Europe/Ljubljana', 'Asia/Bangkok',
       'Americ

In [78]:
def get_rows_by_val(df, col, val, reset_index=True):
    temp_df = df.loc[df[col] == val]
    if reset_index:
        temp_df.reset_index(drop=True, inplace=True)
    return temp_df    

In [79]:
def get_users_by_team(user_df, team):
    temp_df = get_rows_by_val(user_df, 'team', team)
    return temp_df

In [80]:
team_col_idx = 1
print("Reading teams")
steps={}

for i in range(len(step_names)):
    step = {}
    step_name = step_names[i]
    if i == len(step_names)-1:
        next_step = None
    else:
        next_step = step_names[i+1]
    step["next_step"]=next_step

    row_df = steps_df.loc[steps_df['Step'] == step_name]
    team_name = row_df.iat[0,1]
    step["team"] = team_name
    steps[step_name]=step


for s in steps.keys():
    step = steps[s]
    tmp_df = get_users_by_team(users_df,step["team"])
    user_count = len(tmp_df.index)
    print("step : " + s +" team : "+step["team"] + " , #users : "+str(user_count))



Reading teams
step : ColdCall team : Sales , #users : 317
step : Demo & POC team : Pre-Sales , #users : 355
step : Negotiate team : AccountRep , #users : 303
step : Deploy team : CustomerSuccess , #users : 344
step : Test team : QA , #users : 307


In [81]:
steps



{'ColdCall': {'next_step': 'Demo & POC', 'team': 'Sales'},
 'Demo & POC': {'next_step': 'Negotiate', 'team': 'Pre-Sales'},
 'Negotiate': {'next_step': 'Deploy', 'team': 'AccountRep'},
 'Deploy': {'next_step': 'Test', 'team': 'CustomerSuccess'},
 'Test': {'next_step': None, 'team': 'QA'}}

### Generate tasks for step 1 

In [82]:
task_count = 5000
task_id = 0
tasks = []
for i in range(task_count):
    task_type = step_name[0]
    task_tz = random.choice(tz_list)
    task_id += 1
    task = {'task_id':task_id}
    tasks.append(task)


In [83]:
state_of_world = {
    'current_time': datetime.datetime(2005, 1, 1, 1, 0, 0, 0, tzinfo=datetime.timezone.utc),
    'tzs': tz_list,
    'tasks' : steps,
    'users' : users,
    'user_tasks' : {},
    'task_id_cntr' : 0,
    'task_instance_id_cntr' : 0,
}

In [84]:
state_of_world

{'current_time': datetime.datetime(2005, 1, 1, 1, 0, tzinfo=datetime.timezone.utc),
 'tzs': array(['Asia/Colombo', 'Asia/Manila', 'Asia/Harbin', 'Europe/Volgograd',
        'Asia/Jakarta', 'Asia/Shanghai', 'Europe/Stockholm',
        'Asia/Chongqing', 'Europe/Zaporozhye', 'Asia/Yakutsk',
        'Europe/Oslo', 'Asia/Nicosia', 'Asia/Makassar', 'Europe/Paris',
        'Europe/Uzhgorod', 'Europe/Lisbon', 'Europe/Warsaw',
        'America/Guayaquil', 'Atlantic/Cape_Verde', 'Asia/Yekaterinburg',
        'America/Port-au-Prince', 'America/Jamaica', 'America/Lima',
        'Europe/Moscow', 'Asia/Pontianak', 'Asia/Tehran', 'Europe/Tallinn',
        'America/Sao_Paulo', 'America/Mexico_City', 'Asia/Tokyo',
        'Pacific/Auckland', 'America/Mazatlan', 'Africa/Johannesburg',
        'America/Montreal', 'America/Asuncion', 'America/Phoenix',
        'America/Denver', 'Europe/Minsk', 'Asia/Sakhalin',
        'Europe/Amsterdam', 'America/Bogota', 'Asia/Irkutsk',
        'Asia/Almaty', 'America/Mo

In [85]:
def get_users_for_task(task_instance, state):
    task_type = task_instance['task_type']
    task = state['tasks'][task_type]
    team = task['team']
    elig_users = get_users_by_team(state['users'], team)
    return elig_users

In [86]:
def get_next_task_step(current_step):
    if current_step is None:
        step = step_names[0]
    else:
        step = steps[current_step]["next_step"]
    return step

In [104]:



def generate_and_assign_workflow_action(state, prev_inst=None):
    inst={}
    if prev_inst is None:
        next_step = get_next_task_step(None)
        state['task_id_cntr'] += 1
        task_id = state['task_id_cntr']
        prev_task_instance_id = None
    else:
        next_step = get_next_task_step(prev_inst['task_type'])
        task_id = prev_inst['task_id']
        prev_task_instance_id = prev_inst['task_instance_id']
    if next_step is None:
        #Do nothing
        return None
    else:
        inst['prev_task_instance_id'] = prev_task_instance_id
        task_instance_id = state['task_instance_id_cntr']
        state['task_instance_id_cntr'] += 1
        inst['tz'] = random.choice(state['tzs'])
        #Find all users who can execute this task
        inst['task_id'] = task_id
        inst['task_instance_id'] = task_instance_id
        if prev_inst is not None:
            prev_inst['next_task_instance_id'] = task_instance_id
        inst['task_type'] = next_step
        inst['task_type'] = next_step
        elig_users = get_users_for_task(inst , state)
        user_ids = elig_users['id'].unique()
        inst['user_id'] = random.choice(user_ids)
        inst['creation_time_utc'] = state['current_time']
        inst['next_task_instance_id'] = None
        inst['start_time_utc'] = None
        inst['stop_time_utc'] = None
        return inst



In [88]:
state = state_of_world

In [105]:
task = None
task_instances = []
for i in range(100):
    task = generate_and_assign_workflow_action(state,task)
    if task is not None:
        task_instances.append(task)

In [106]:

t_inst_df = pd.DataFrame(task_instances)

In [107]:
t_inst_df


,prev_task_instance_id,tz,task_id,task_instance_id,task_type,user_id,creation_time_utc,next_task_instance_id,start_time_utc,stop_time_utc
0,NaN,Pacific/Auckland,41,174,ColdCall,565,2005-01-01 01:00:00+00:00,175.0,None,None
1,174.0,Europe/Luxembourg,41,175,Demo & POC,333,2005-01-01 01:00:00+00:00,176.0,None,None
2,175.0,America/Costa_Rica,41,176,Negotiate,13,2005-01-01 01:00:00+00:00,177.0,None,None
3,176.0,Indian/Maldives,41,177,Deploy,170,2005-01-01 01:00:00+00:00,178.0,None,None
4,177.0,Asia/Dushanbe,41,178,Test,468,2005-01-01 01:00:00+00:00,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...
79,252.0,America/Monterrey,56,253,Test,355,2005-01-01 01:00:00+00:00,NaN,None,None
80,NaN,America/Guadeloupe,57,254,ColdCall,149,2005-01-01 01:00:00+00:00,255.0,None,None
81,254.0,Atlantic/Cape_Verde,57,255,Demo & POC,856,2005-01-01 01:00:00+00:00,256.0,None,None
82,255.0,Asia/Ulaanbaatar,57,256,Negotiate,852,2005-01-01 01:00:00+00:00,257.0,None,None


In [100]:
task = None
task = generate_and_assign_workflow_action(state,task)
task


{'task_id': 23,
 'task_instance_id': 89,
 'task_type': 'ColdCall',
 'user_id': 414,
 'creation_time_utc': datetime.datetime(2005, 1, 1, 1, 0, tzinfo=datetime.timezone.utc),
 'next_task_instance_id': None,
 'start_time_utc': None,
 'stop_time_utc': None}